# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)

Since it was tried to keep things modular and save on install time, you need to pick on what to install yourself.

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.10.12
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#@title download all dependencies (~20min)
#tensorrt
%cd /content
!sudo apt-get update
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer tensorrt python3-libnvinfer-dev

# precompiled
%cd /content
!sudo rm -rf /workspace
!mkdir -p /workspace/tensorrt
%cd /workspace/tensorrt
!git clone https://github.com/styler00dollar/VSGAN-tensorrt-docker

!mkdir vs_precompiled_colab
%cd vs_precompiled_colab
!wget https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/vs_precompiled_colab_py310.7z
!7z x vs_precompiled_colab_py310.7z -O/content/vs_precompiled_colab_py310/

!mkdir -p /usr/local/lib/vapoursynth/
!mkdir -p /usr/local/lib/x86_64-linux-gnu/
!mkdir -p /usr/local/lib/x86_64-linux-gnu/vapoursynth/
!mkdir -p /usr/local/lib/python3.10/site-packages/

!mv /content/vs_precompiled_colab_py310/vspipe /usr/local/bin/vspipe
!mv /content/vs_precompiled_colab_py310/libwwxd.so /usr/local/lib/libwwxd.so
!mv /content/vs_precompiled_colab_py310/libawarpsharp2.so /usr/local/lib/x86_64-linux-gnu/libawarpsharp2.so
!mv /content/vs_precompiled_colab_py310/libfmtconv.so /usr/local/lib/libfmtconv.so
!mv /content/vs_precompiled_colab_py310/libakarin.so /usr/local/lib/vapoursynth/libakarin.so
!mv /content/vs_precompiled_colab_py310/libjulek.so /usr/local/lib/vapoursynth/libjulek.so
!mv /content/vs_precompiled_colab_py310/librife.so /usr/local/lib/vapoursynth/librife.so
!mv /content/vs_precompiled_colab_py310/libmvtools.so /usr/local/lib/libmvtools.so
!mv /content/vs_precompiled_colab_py310/libdescale.so /usr/local/lib/vapoursynth/libdescale.so
!mv /content/vs_precompiled_colab_py310/libvfrtocfr.so /usr/local/lib/x86_64-linux-gnu/vapoursynth/libvfrtocfr.so
!mv /content/vs_precompiled_colab_py310/libvapoursynth.so /usr/local/lib/libvapoursynth.so
!mv /content/vs_precompiled_colab_py310/libvapoursynth-script.so /usr/local/lib/libvapoursynth-script.so
!mv /content/vs_precompiled_colab_py310/vapoursynth_dyn.so /usr/lib/python3.10/lib-dynload/vapoursynth.so
!mv /content/vs_precompiled_colab_py310/vapoursynth.so /usr/local/lib/python3.10/site-packages/vapoursynth.so
!mv /content/vs_precompiled_colab_py310/libzimg.so /usr/local/lib/libzimg.so
!mv /content/vs_precompiled_colab_py310/libvstrt.so /usr/local/lib/libvstrt.so
!mv /content/vs_precompiled_colab_py310/libmiscfilters.so /usr/local/lib/vapoursynth/libmiscfilters.so
!mv /content/vs_precompiled_colab_py310/libvslsmashsource.so /usr/local/lib/vapoursynth/libvslsmashsource.so
!mv /content/vs_precompiled_colab_py310/libvmaf_vs.so /usr/local/lib/vapoursynth/libvmaf.so
!mv /content/vs_precompiled_colab_py310/libvmaf.so /usr/local/lib/x86_64-linux-gnu/libvmaf.so
!mv /content/vs_precompiled_colab_py310/libbestsource.so /usr/local/lib/vapoursynth/libbestsource.so
!mv /content/vs_precompiled_colab_py310/liblsmash.so.2 /usr/local/lib/liblsmash.so.2
!mv /content/vs_precompiled_colab_py310/libvulkan.so /usr/lib/x86_64-linux-gnu/libvulkan.so

!sudo ldconfig
!chmod +x /usr/local/bin/vspipe

# lsmash
!sudo apt -y install libjansson-dev libzimg-dev nasm ninja-build
!pip install ninja meson
%cd /content
!git clone https://code.videolan.org/videolan/dav1d/ && \
  cd dav1d && meson build --buildtype release --default-library static && ninja -C build install
%cd /content
# FFmpeg/FFmpeg: 40aa451154fc54b661036bfb90532199269397bc
!git clone https://github.com/FFmpeg/FFmpeg && cd FFmpeg && \
  CFLAGS=-fPIC ./configure --enable-shared --disable-static --enable-pic --enable-gpl --enable-version3 --disable-programs --disable-doc --disable-avdevice \
  --disable-swresample --disable-postproc --disable-avfilter --disable-encoders --disable-muxers \
  --enable-libdav1d --disable-nvenc --disable-debug && \
  make -j$(nproc) && make install && ldconfig && cd /content && rm -rf FFmpeg

# python
!pip install numpy && pip install docutils pygments && pip install git+https://github.com/hahnec/color-matcher
!pip install /content/vs_precompiled_colab_py310/*.whl kornia opencv-python pytorch-msssim thop einops timm wget mpgg vsutil \
  git+https://github.com/styler00dollar/vs-gmfss_union git+https://github.com/styler00dollar/vs-gmfss_fortuna git+https://github.com/styler00dollar/vs-dpir \
  scenedetect onnxruntime-gpu vsbasicvsrpp vsswinir https://github.com/pytorch/TensorRT/releases/download/v1.4.0/torch_tensorrt-1.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install torch-tensorrt-fx-only==1.5.0.dev0 && pip install nvidia-pyindex tensorrt==8.6.1 && pip install polygraphy

# rife
!sudo apt-get install libvulkan-dev libvulkan1 mesa-vulkan-drivers vulkan-tools -y
!apt-get install libnvidia-gl-525 -y
%cd /content
!git clone https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
%cd VapourSynth-RIFE-ncnn-Vulkan
!mv models /usr/local/lib/vapoursynth/

# upgrading g++ and installing ffms2
!sudo apt install build-essential manpages-dev software-properties-common ffmsindex libffms2-5 libffms2-dev -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

# encoder
!sudo apt install x264 x265 -y

# Render

# Normal inference

In [ ]:
# if you want to use vs-mlrt, convert model into engine, some onnx are here: https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/tag/models
# after creating the model, you can set the path in inference.py
%cd /content
!/usr/src/tensorrt/bin/trtexec --fp16 --onnx=model.onnx --minShapes=input:1x3x8x8 --optShapes=input:1x3x720x1280 --maxShapes=input:1x3x1080x1920 --saveEngine=/content/model.engine --tacticSources=+CUDNN,+CUBLAS,+CUBLAS_LT

In [ ]:
#@title inference.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference.py
import sys

sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
from inference_config import inference_clip

video_path = "/content/test.webm"
clip = inference_clip(video_path)
clip.set_output()

In [ ]:
#@title inference_config.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference_config.py
import sys
import os

sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
import vapoursynth as vs

# video imports
from src.vfi_inference import vfi_inference
from src.vfi_model import video_model

from src.rife import RIFE
from src.IFRNet import IFRNet
from src.GMFupSS import GMFupSS
from src.GMFSS_union import GMFSS_union
from vsgmfss_union import gmfss_union
from src.eisai import EISAI
from src.film import FILM
from src.M2M import M2M
from src.sepconv_enhanced import sepconv
from src.IFUNet import IFUNet
from src.stmfnet import STMFNet
from src.rife_trt import rife_trt
from src.cain_trt import cain_trt
from src.GMFSS_Fortuna_union import GMFSS_Fortuna_union
from src.GMFSS_Fortuna import GMFSS_Fortuna
from vsgmfss_fortuna import gmfss_fortuna
from vsdpir import dpir

# upscale imports
from vsbasicvsrpp import basicvsrpp
from vsswinir import SwinIR

from src.scene_detect import scene_detect

from src.color_transfer import vs_color_match

core = vs.core
vs_api_below4 = vs.__api_version__.api_major < 4
core = vs.core
core.num_threads = 4  # can influence ram usage
# only needed if you are inside docker
core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libvstrt.so")
core.std.LoadPlugin(path="/usr/local/lib/x86_64-linux-gnu/libawarpsharp2.so")


def inference_clip(video_path="", clip=None):
    # ddfi is passing clip
    if clip is None:
        # cfr video
        #clip = core.ffms2.Source(source=video_path, cache=False)
        # vfr video
        # clip = core.ffms2.Source(source=video_path, fpsnum = 24000, fpsden = 1001, cache=False)
        # vfr video (automatically set num and den)
        # clip = core.ffms2.Source(source=video_path, fpsnum = -1, fpsden = 1, cache=False)

        # lsmash
        clip = core.lsmas.LWLibavSource(source=video_path)
        # lsmash with hw decoding preferred
        # clip = core.lsmas.LWLibavSource(source=video_path, prefer_hw=3)

        # resizing with descale
        # Debilinear, Debicubic, Delanczos, Despline16, Despline36, Despline64, Descale
        # clip = core.descale.Debilinear(clip, 1280, 720)

    ###############################################
    # SIMILARITY
    # Set properties in clip for it to be applied
    # SSIM for deduplication in frame interpolation

    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # 0 = PSNR, 1 = PSNR-HVS, 2 = SSIM, 3 = MS-SSIM, 4 = CIEDE2000
    # clip = core.vmaf.Metric(clip, offs1, 2)

    # SCENE DETECT
    # clip = core.misc.SCDetect(clip=clip, threshold=0.100)
    # model based scene detect
    # clip = scene_detect(clip, model_name="efficientnetv2_b0", thresh=0.98, fp16=False)

    # dedup (requires you to call "vspipe parse.py -p ." to generate infos_running.txt and tmp.txt)
    # from src.dedup import get_dedup_frames
    # frames_duplicated, frames_duplicating = get_dedup_frames()
    # clip = core.std.DeleteFrames(clip, frames_duplicated)
    # do upscaling here
    # clip = core.std.DuplicateFrames(clip, frames_duplicating)
    ###############################################
    # COLORSPACE
    ###############################################

    # convert colorspace
    clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")
    # clip = vs.core.resize.Spline64(clip, format=vs.RGBS, matrix_in_s="709", transfer_in_s="linear")

    # convert colorspace + resizing
    # clip = vs.core.resize.Bicubic(
    #    clip, width=1280, height=720, format=vs.RGBS, matrix_in_s="709"
    # )

    ###############################################
    # MODELS
    ###############################################
    # in rare cases it can happen that image range is not 0-1 and that resulting in big visual problems, clamp input
    # clip = core.akarin.Expr(clip, "x 0 1 clamp")
    # clip = clip.std.Expr("x 0 max 1 min")
    # clip = core.std.Limiter(clip, max=1, planes=[0,1,2])

    ######
    # VFI
    ######

    # VFI example for jit models
    # clip = video_model(clip, fp16=False, model_path="/workspace/rvpV1_105661_G.pt")

    # Rife: model "rife40" up to "rife46" and "sudo_rife4"
    # model_inference = RIFE(
    #    scale=1, fastmode=True, ensemble=False, model_version="rife46", fp16=True
    # )

    # IFRNet: model="small" or "large"
    # model_inference = IFRNet(model="small", fp16=False)

    # use gmfss_union instead for more speed
    # model_inference = GMFupSS(partial_fp16=False)
    # model_inference = GMFSS_union(partial_fp16=False)

    # model_inference = EISAI() # 960x540

    # FILM: model_choise="style", "l1" or "vgg"
    # model_inference = FILM(model_choise="vgg")

    # model_inference = M2M()

    # model_inference = sepconv() # only 2x supported because architecture only outputs one image

    # model_inference = IFUNet()

    # model_inference = STMFNet()  # only 2x supported because architecture only outputs one image

    # model_inference = GMFSS_Fortuna_union()

    # model_inference = GMFSS_Fortuna()

    # clip = vfi_inference(
    #    model_inference=model_inference, clip=clip, multi=2, metric_thresh=0.999
    # )

    # clip = rife_trt(clip, multi = 2, scale = 1.0, device_id = 0, num_streams = 2, engine_path = "/workspace/tensorrt/rife46.engine")

    # clip = cain_trt(clip, device_id = 0, num_streams = 4, engine_path = "/workspace/tensorrt/rvp.engine")

    # clip = gmfss_union(clip, num_streams=4, trt=True, factor_num=2, ensemble=False, sc=True, trt_cache_path="/workspace/tensorrt/")

    # clip = gmfss_union(clip, num_streams=4, trt=True, factor_num=2, ensemble=False, sc=True, trt_cache_path="/workspace/tensorrt/")

    # more information here: https://github.com/HolyWu/vs-gmfss_fortuna/blob/master/vsgmfss_fortuna/__init__.py
    # clip = gmfss_fortuna(clip, num_streams=4, trt=True, factor_num=2, factor_den=1, model=1, ensemble=False, sc=True, trt_cache_path="/workspace/tensorrt/",)

    ######
    # UPSCALING WITH TENSORRT
    ######
    # vs-mlrt (you need to create the engine yourself, read the readme)
    #clip = core.trt.Model(
    #    clip,
    #    engine_path="/content/model.engine",
    #    # tilesize=[854, 480],
    #    overlap=[0, 0],
    #    num_streams=2,
    #)

    # vs-mlrt (DPIR)
    # DPIR does need an extra channel
    # strength = 10.0
    # noise_level = clip.std.BlankClip(format=vs.GRAYS, color=strength / 100)
    # clip = core.trt.Model(
    #    [clip, noise_level],
    #    engine_path="dpir.engine",
    #    tilesize=[1280, 720],
    #    num_streams=2,
    # )

    ######
    # external vs plugins
    ######

    # BasicVSR++
    # model list: https://github.com/HolyWu/vs-basicvsrpp/blob/0ad97ca908707d883922f092428337972a8d0215/vsbasicvsrpp/__init__.py#L42
    # clip = basicvsrpp(clip, model = 1, length = 15, cpu_cache = False, tile_w = 0, tile_h = 0, tile_pad = 16)

    # SwinIR
    # clip = SwinIR(clip, task="lightweight_sr", scale=2)

    ###############################################
    # ncnn (works in docker, but only on linux, because wsl on windows does not support vulkan)
    ###############################################

    # Rife ncnn (C++)
    # Model list can be found in https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
    clip = core.misc.SCDetect(clip=clip, threshold=0.100)
    clip = core.rife.RIFE(
        clip,
        model=9,
        factor_num=2,
        gpu_id=0,
        gpu_thread=1,
        tta=False,
        uhd=False,
        skip=True,
        sc=True,
    )

    ######
    # DDFI
    # you need to use 8x interp for this
    ######
    # advanced example with pytorch vfi + dedup + scene change + upscaling

    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # clip = core.vmaf.Metric(clip, offs1, 2)
    # clip = core.resize.Bicubic(clip, width=1280, height=720, format=vs.RGBS, matrix_in=1)

    # clip = core.misc.SCDetect(clip=clip, threshold=0.100)

    # model_inference = GMFupSS(partial_fp16=True)
    # clip = vfi_inference(
    #     model_inference=model_inference, clip=clip, multi=8, metric_thresh=0.999
    # )

    # clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
    # offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
    # offs1 = core.std.CopyFrameProps(offs1, clip)
    # clip = core.vmaf.Metric(clip, offs1, 2)
    # clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")

    # clip = core.trt.Model(
    #     clip,
    #     engine_path="/content/model.engine",
    #     num_streams=3,
    # )

    ####
    # Color Transfer
    ####

    # original_clip = clip
    # original_clip = original_clip.resize.Spline16(format=vs.RGB24, matrix_in_s="470bg")
    # clip = clip.resize.Spline16(format=vs.RGB24, matrix_in_s="470bg")
    # clip = vs_color_match(clip, original_clip, method="mkl")

    ###
    # Other
    ###
    # does not accept rgb clip, convert to yuv first
    # clip = core.warp.AWarpSharp2(clip, thresh=128, blur=2, type=0, depth=[16, 8, 8], chroma=0, opt=True, planes=[0,1,2], cplace="mpeg1")

    # more information here: https://github.com/HolyWu/vs-dpir/blob/master/vsdpir/__init__.py
    # clip = dpir(clip, num_streams = 4, nvfuser = False, cuda_graphs = False, trt = True, trt_cache_path = "/workspace/tensorrt/", task = "deblock", strength = 50, tile_w = 0, tile_h = 0, tile_pad= 8)

    # clip = core.cas.CAS(clip, sharpness=0.5)

    ###############################################
    # OUTPUT
    ###############################################
    clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
    return clip

In [ ]:
# default ubuntu ffmpeg (outdated ffmpeg)
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | ffmpeg -i pipe: /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | x264 - --demuxer y4m -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | x265 - --y4m --crf 23 -o /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv -y

# Batch inference
Also uses `inference_config.py` for configuration, but you also need to use `main.py` to iterate over files.

In [ ]:
#@title main.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/main.py
import glob
import os
import shutil

input_dir = "/workspace/tensorrt/input/"
tmp_dir = "tmp/"
output_dir = "/workspace/tensorrt/output/"
files = glob.glob(input_dir + "/**/*.mkv", recursive=True)
files.sort()

for f in files:
    # creating folders if they dont exist
    if not os.path.exists(tmp_dir):
        os.mkdir(tmp_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if os.path.exists(tmp_dir):
        shutil.rmtree(tmp_dir)
        os.mkdir(tmp_dir)

    # paths
    out_render_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_rendered.mkv"
    )
    mux_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_mux.mkv"
    )

    # only needed for dedup
    # os.system(f"vspipe /workspace/tensorrt/parse.py --arg source='{f}' -p .")

    # os.system(f"vspipe -c y4m inference_batch.py - | ffmpeg -i pipe: {out_render_path}")
    # os.system(
    #    f"ffmpeg -y -loglevel error -i '{f}' -i {out_render_path}  -map 1 -map 0 -map -0:v -codec copy -max_interleave_delta 0 '{mux_path}'"
    # )

    ###### example presets ######
    # speeds are resizised 4k footage with 7950x
    # in terms of color banding:
    # (less) x264 >  libx265 > libaom-av1 (with grain) > libsvtav1 > av1_nvenc (more)
    # new encoders might result in smaller filesize/quality ratio, but there are banding problems which persist even with higher quality settings though
    # in my personal opinion, stick with x264 for no banding and speed and aom for quality for filesize

    ### slower ###
    # x265 crf10 preset slow
    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -vcodec libx265 -crf 10 -preset slow '{mux_path}'"
    # )

    # aom av1 (for quality/filesize but slow, encoder has banding issues without grain table)
    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -vcodec libaom-av1 -cpu-used 6 -lag-in-frames 48 -arnr-max-frames 4 -arnr-strength 1 -enable-cdef 1 -enable-restoration 0 -tune ssim -aom-params film-grain-table='/workspace/tensorrt/grain.tbl',input-depth=10,fp-mt=1,keyint=240 -crf 10 -tile-columns 1 -tile-rows 1 -row-mt 1 '{mux_path}'"
    # )

    ### medium ###
    # x265 crf10 default preset
    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -vcodec libx265 -crf 10 '{mux_path}'"
    # )

    ### faster ###
    # x264 crf10 default preset
    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -crf 10 '{mux_path}'"
    # )

    # x264 crf10 preset slow
    os.system(
        f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -crf 10 -preset slow '{mux_path}'"
    )

    # svt av1 (encoder has banding issues)
    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i '{f}' -thread_queue_size 100 -i pipe: -map 1 -map 0 -map -0:v -max_interleave_delta 0 -scodec copy -vcodec libsvtav1 -svtav1-params tune=0,enable-overlays=1,enable-qm=1 -preset 8 -crf 10 '{mux_path}'"
    # )

    # os.system(
    #   f"vspipe -c y4m inference_batch.py --arg source='{f}' - | ffmpeg -y -i pipe: %05d.png"
    # )

    # deleting temp files
    # os.remove(txt_path)

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker
!python main.py

# ddfi

Auto dedup-duplication inference example (More information on what exactly is meant is in [Mr-Z-2697/ddfi-rife](https://github.com/Mr-Z-2697/ddfi-rife).) Also uses `inference_config.py` for configuration, but you also need to use `deduped_vfi.py` to iterate over files.

In [ ]:
#@title parse.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/parse.py
import vapoursynth as vs
import os
import functools

core = vs.core
core.num_threads = 4
core.max_cache_size = 4096

core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libfmtconv.so")
core.std.LoadPlugin(path="/usr/local/lib/libmvtools.so")

# https://github.com/xyx98/my-vapoursynth-script/blob/master/xvs.py
def props2csv(
    clip: vs.VideoNode,
    props: list,
    titles: list,
    output="info.csv",
    sep="\t",
    charset="utf-8",
    tostring=None,
):
    file = open(output, "w", encoding=charset)
    file.write(sep.join(["n"] + titles))
    tostring = (
        tostring
        if callable(tostring)
        else lambda x: x.decode("utf-8")
        if isinstance(x, bytes)
        else str(x)
    )

    def tocsv(n, f, clip):
        file.write(
            "\n"
            + sep.join(
                [str(n)]
                + [tostring(eval("f.props." + i, globals(), {"f": f})) for i in props]
            )
        )

        return clip
        file.close()

    return core.std.FrameEval(clip, functools.partial(tocsv, clip=clip), prop_src=clip)


clip = core.ffms2.Source(globals()["source"], cache=False)
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
offs1 = core.vmaf.Metric(clip, offs1, 2)
offs1 = core.std.MakeDiff(offs1, clip)
offs1 = core.fmtc.bitdepth(offs1, bits=16)
offs1 = core.std.Expr(offs1, "x 32768 - abs")
offs1 = core.std.PlaneStats(offs1)
offs1 = props2csv(
    offs1,
    props=["_AbsoluteTime", "float_ssim", "PlaneStatsMax"],
    output=os.path.join(tmp_dir, "infos_running.txt"),
    titles=[],
)
offs1.set_output()

In [ ]:
#@title ddfi.py
#@markdown edit this file to customize interpolation
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/ddfi.py
import sys
sys.path.append("/workspace/tensorrt/VSGAN-tensorrt-docker/")
import vapoursynth as vs
import os
from src.rife import RIFE
from src.IFRNet import IFRNet
from src.GMFupSS import GMFupSS
from src.eisai import EISAI
from src.film import FILM
from src.vfi_inference import vfi_inference

core = vs.core

core.std.LoadPlugin(path="/usr/lib/x86_64-linux-gnu/libffms2.so")
core.std.LoadPlugin(path="/usr/local/lib/libmvtools.so")
core.std.LoadPlugin(path="/usr/local/lib/libvstrt.so")


ssimt = 0.999
pxdifft = 10240
consecutivet = 2
core.num_threads = 4
core.max_cache_size = 4096

tmp_dir = "tmp/"

# frames to delete
def processInfo():
    with open(os.path.join(tmp_dir, "infos_running.txt"), "r") as f:
        lines = [i.split("\t") for i in f][1:]
    for i in range(len(lines)):
        lines[i][0] = int(lines[i][0])
        lines[i][1] = int(float(lines[i][1]) * 1000)
        lines[i][2] = float(lines[i][2])
        lines[i][3] = int(lines[i][3])
    lines.sort()
    startpts = lines[0][1]
    consecutive = 0

    dels = []
    tsv2o = []
    for i in range(len(lines)):
        l = lines[i]
        if l[2] >= ssimt and l[3] <= pxdifft and consecutive < consecutivet:
            consecutive += 1
            dels.append(l[0])
        else:
            consecutive = 0
            tsv2o.append(l[1] - startpts)
    return dels, tsv2o


def newTSgen(tsv2o):
    ts_new = list()
    outfile = open(os.path.join(tmp_dir, "tsv2nX8.txt"), "w", encoding="utf-8")
    ts_o = [i for i in tsv2o][1:]

    for x in range(len(ts_o) - 1):
        ts_new.append(str(float(ts_o[x])))
        for i in range(1, 8):
            ts_new.append(
                str(float(ts_o[x]) + (float(ts_o[x + 1]) - float(ts_o[x])) / 8 * i)
            )
    print("#timestamp format v2", file=outfile)
    for x in range(len(ts_new)):
        print(ts_new[x], file=outfile)
    print(ts_o[len(ts_o) - 1], file=outfile)
    outfile.close()


dels, tsv2o = processInfo()
newTSgen(tsv2o)

with open(os.path.join(tmp_dir, "tmp.txt")) as f:
    video_path = f.readlines()[0]
clip = core.ffms2.Source(video_path)
clip = core.std.DeleteFrames(clip, dels)
sup = core.mv.Super(clip, pel=1, levels=1)
bw = core.mv.Analyse(sup, isb=True, levels=1, truemotion=False)
clip = core.mv.SCDetection(clip, bw, thscd1=200, thscd2=85)


# easy example with ncnn rife
clip = core.resize.Bicubic(clip, format=vs.RGBS, matrix_in=1)
clip = core.misc.SCDetect(clip=clip, threshold=0.100)
clip = core.rife.RIFE(clip, model=9, sc=True, skip=True, multiplier=8)

# example for custom vfi
"""
clip = core.resize.Bicubic(clip, format=vs.RGBS, matrix_in=1)

model_inference = RIFE(
    scale=1, fastmode=False, ensemble=True, model_version="rife46", fp16=False
)
# model_inference = IFRNet(model="small", fp16=False)
# model_inference = GMFupSS()
# model_inference = EISAI() # 960x540
# model_inference = FILM(model_choise="vgg")
clip = vfi_inference(
    model_inference=model_inference, clip=clip, multi=8
)
"""

# advanced example with pytorch vfi + dedup + scene change + upscaling
"""
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
clip = core.vmaf.Metric(clip, offs1, 2)
clip = core.resize.Bicubic(clip, width=1280, height=720, format=vs.RGBS, matrix_in=1)

clip = core.misc.SCDetect(clip=clip, threshold=0.100)

model_inference = GMFupSS(partial_fp16=True)
clip = vfi_inference(
    model_inference=model_inference, clip=clip, multi=8
)

clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
offs1 = core.std.BlankClip(clip, length=1) + clip[:-1]
offs1 = core.std.CopyFrameProps(offs1, clip)
clip = core.vmaf.Metric(clip, offs1, 2)
clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s="709")

clip = core.trt.Model(
    clip,
    engine_path="/content/model.engine",
    num_streams=3,
)
"""
#######################

clip = core.resize.Bicubic(
    clip, format=vs.YUV420P10, matrix=1, dither_type="error_diffusion"
)
clip = core.vfrtocfr.VFRToCFR(
    clip, os.path.join(tmp_dir, "tsv2nX8.txt"), 192000, 1001, True
)  # 24fps * 8
sup = core.mv.Super(clip)
fw = core.mv.Analyse(sup)
bw = core.mv.Analyse(sup, isb=True)
clip = core.mv.FlowFPS(clip, sup, bw, fw, 60, 1)
clip.set_output()

In [ ]:
#@title deduped_vfi.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/deduped_vfi.py
import glob
import os

input_dir = "/workspace/tensorrt/VSGAN-tensorrt-docker/input/"
tmp_dir = "tmp/"
output_dir = "/workspace/tensorrt/VSGAN-tensorrt-docker/output/"
files = glob.glob(input_dir + "/**/*.mkv", recursive=True)
files.sort()

for f in files:
    # creating folders if they dont exist
    if os.path.exists(tmp_dir) == False:
        os.mkdir(tmp_dir)
    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    # paths
    txt_path = os.path.join(tmp_dir, "tmp.txt")
    out_render_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_rendered.mkv"
    )
    mux_path = os.path.join(
        output_dir, os.path.splitext(os.path.basename(f))[0] + "_mux.mkv"
    )

    # writing filepath into temp txt
    # workaround to pass filename parameter
    f_txt = open(txt_path, "w")
    f_txt.write(str(f))
    f_txt.close()

    os.system("vspipe parse.py -p .")
    os.system(
        f"vspipe -c y4m ddfi.py - | ffmpeg -i pipe: -vcodec libsvtav1 -crf 20 {out_render_path}"
    )

    os.system(
        f"ffmpeg -y -loglevel error -i {f} -i {out_render_path}  -map 1 -map 0 -map -0:v -codec copy -max_interleave_delta 0 {mux_path}"
    )

    # deleting temp files
    os.remove(txt_path)

In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!python deduped_vfi.py